### Probabilistic PCA for TB EBA trial biomarkers

### import modules

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from ppca import PPCA
import matplotlib.cm as cm

np.random.seed(42)

## Read data

In [ ]:
### Data matrix must be in long format, single row per timepoint, with at least 1 non-na value per row
### Columns == ID, timepoint, logCFU, logTTP, logMBLA, Treatment_group [A,B,C]

X=pd.read_csv('./data.csv')

## pPCA 

### Pre model checks
> Check if data is stationary

In [ ]:
# Run ADF test per assay
pval0=adfuller(X['logCFU'].dropna())[1] 
pval1=adfuller(X['logTTP'].dropna())[1] 
pval2=adfuller(X['logMBLA'].dropna())[1] 

print(f'P-value for CFU H0 test {pval0}')
print(f'P-value for MGIT TTP H0 test {pval1}')
print(f'P-value for TB-MBLA H0 test {pval2}')
## If data is stationary (P<alpha)) - OK to proceed

### pPCA

In [ ]:
pca_df = X.drop(columns=['Treatment_group'])
cols = ['logCFU','logTTP','logMBLA']
subset_df = pca_df[cols]
cs_df = (subset_df - subset_df.mean(0)) / subset_df.std(0) # centre/scale
pca_df[cols] = cs_df
pca_df = pca_df.set_index(['ID','timepoint'])
pca_df.dropna(axis=0, subset=['logCFU','logTTP','logMBLA'], how='all', inplace=True) # drop instances of no values

In [ ]:
ppca = PPCA()
ppca.fit(pca_df.values, d=2) # d== n components/PCs desired

In [ ]:
variance_explained = ppca.var_exp ### Variance explained per PC [PC1, PC2, etc]
components = ppca.data 
model_params = ppca.C # loadings (assays x pcs)
component_mat = ppca.transform() # get components
pca_df['pc1'] = component_mat[:,0] # add PC1 to df
pca_df['pc2'] = component_mat[:,1] # add PC2 to df

In [ ]:
ppca.save('mypcamodel_example') 
#ppca.load('mypcamodel_example.npy')

In [ ]:
pca_df=pca_df.reset_index()

In [ ]:
pca_df_full = pd.merge(pca_df, X[['ID','timepoint', 'Treatment_group']], on=['ID','timepoint'], how='left')

### Plot

In [ ]:
ppca.C.T

In [ ]:
fig , ax = plt.subplots(1, 1, figsize=(7,7))

sns.scatterplot(
    x='pc1', y='pc2', 
    data=pca_df_full, 
    s=50, 
    facecolor='white',
    edgecolor='black',
    linewidth=1,
    alpha=0.4,
    ax=ax
)

i, j = 0, 1 # which components

ax.set_xlabel('PC%d' % (i+1),fontsize=20)
ax.set_ylabel('PC%d' % (j+1),fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)

assays = pca_df_full[['Assay_1','logTTP','logMBLA']] 


### Amend to suit case                    
offsets = {
    'logCFU': (0.01, 0.02),
    'logTTP': (-0.04, 0.03),
    'logMBLA': (0.02, -0.1)
}

for k in range(ppca.C.T.shape[1]):
    x = ppca.C.T[i,k]
    y = ppca.C.T[j,k]
    label = assays.columns[k]
    
    dx, dy = offsets.get(label, (0, 0))  # Default to no offset
    
    ax.arrow(0, 0, x, y)
    ax.text(x + dx, y + dy, label, fontsize=20, fontweight="bold", color='red')

#plt.savefig('ppcc_biplot.tiff', bbox_inches='tight',format='tiff',dpi=400)
plt.show()

#### Write out (if required)

In [ ]:
pca_df_full.to_csv("./data.csv", index=False)